###### Imports and Settings

In [2]:
import pandas as pd
import numpy as np
import requests
from functools import reduce
import matplotlib.pyplot as plt
import pickle
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
import sys
sys.path.append("../../../Functions and Dictionaries") # Adds higher directory to python modules path
import geodict
tofullcensus = geodict.tofullcensus
geotogeoid = geodict.geotogeoid
import sqlite3 as sq

# This notebook outlines the download and formatting process for data from the Tennessee Bureau of Investigation.  

Go to this page: https://crimeinsight.tbi.tn.gov/public/View/dispview.aspx?ReportId=20, "Crime Rates by Jurisdiction" fom this page: https://crimeinsight.tbi.tn.gov/public/Browse/browsetables.aspx
We are looking for Crime Rates by county and the State of Tennessee here. Select "Jurisdiction by Geography" on the left-side of the screen, where you are then able to manipulate geographies by level. Select the State, and then deselect the regions, select counties in the GNRC operating area (most in Middle, some in West), and under those counties leave the jurisdictions. Click "show report" in the top right-side of the screen, and then download a multi-dimensional csv file. Delete the data headers, rename Jurisdiction by Geography to "NAME", make sure that the numbers are not formatted with commas, add a county column to tag the juridictions with a county, leaving the column null for the state and counties, and set the column headers correctly before saving the file as "TBRCrimeRatesbyJurisdiction_IncidentDate(YYYY).csv.  

Next, From the homepage select "Crime and Clearance by Offense Type", deselect clearance in the left-side selector. Select "Jurisdiction by Geography" and similarly select the State and all Counties and jurisdictions. Download this, format, transpose data so NAME is on the left and you can create a similar county column as before, and save as 

*Note*: the jurisdictions don't necessarily repond to a place level plan in all cases, but they may and you can toy around here to see if they match the plan or document you're generating data for!


In [111]:
# #import and examine data
# data = pd.read_csv('../../Data Downloads/TBI_CrimeRatesbyJurisdiction_IncidentDate2021.csv')
# data['Year'] = '2021'
# data.head(3)
#import and examine data
data = pd.read_csv('../../Data Downloads/TBI_CrimeRatesbyJurisdiction_IncidentDate2022.csv')
#data['Year'] = '2022'
data.head(3)

,NAME,County,"Crime Rate (per 1,000)",Estimated Population,Number of Crimes
0,Tennessee,Full,71.89,7051813.0,506962.0
1,Cheatham,Full,43.94,42080.0,1849.0
2,Cheatham County Sheriff's Office,Cheatham,NaN,NaN,1068.0


In [112]:
#create a dictionary to rename the counties to full census names
countydict = {'Cheatham': 'Cheatham County, Tennessee', 'Davidson': 'Davidson County, Tennessee', 'Maury': 'Maury County, Tennessee', 
              'Robertson': 'Robertson County, Tennessee', 'Rutherford': 'Rutherford County, Tennessee', 'Sumner': 'Sumner County, Tennessee', 
              'Trousdale': 'Trousdale County, Tennessee', 'Williamson': 'Williamson County, Tennessee', 'Wilson': 'Wilson County, Tennessee', 
              'Dickson': 'Dickson County, Tennessee', 'Houston': 'Houston County, Tennessee', "Humphreys": 'Humphreys County, Tennessee',
              'Montgomery': 'Montgomery County, Tennessee', 'Stewart': 'Stewart County, Tennessee'}

In [113]:
data = data.set_index('County').transpose()
data = data.rename(columns = countydict)
data = data.transpose().reset_index()

In [114]:
aside = data.loc[data['County'] != 'Full']
data = data.loc[data['County'] == 'Full'].drop(columns = 'County')

In [115]:
data.head()

,NAME,"Crime Rate (per 1,000)",Estimated Population,Number of Crimes
0,Tennessee,71.89,7051813.0,506962.0
1,Cheatham,43.94,42080.0,1849.0
6,Davidson,115.77,694520.0,80404.0
50,Maury,67.91,108210.0,7349.0
57,Robertson,37.27,75565.0,2816.0


In [116]:
#transpose to aggregate regions, then transpose back
data = data.set_index('NAME').transpose()
data = data.rename(columns = countydict)
GNRCCounties = [data['Stewart County, Tennessee'],data['Montgomery County, Tennessee'],
                data['Houston County, Tennessee'],data['Humphreys County, Tennessee'],
                data['Dickson County, Tennessee'],data['Cheatham County, Tennessee'],
                data['Robertson County, Tennessee'],data['Sumner County, Tennessee'],
                data['Davidson County, Tennessee'],data['Wilson County, Tennessee'],
                data['Trousdale County, Tennessee'],data['Williamson County, Tennessee'],
                data['Rutherford County, Tennessee']]
data['GNRC'] = sum(GNRCCounties)
GNRCCountiesAll = [data['Stewart County, Tennessee'],data['Montgomery County, Tennessee'],
                   data['Houston County, Tennessee'],data['Humphreys County, Tennessee'],
                   data['Dickson County, Tennessee'],data['Cheatham County, Tennessee'],
                   data['Robertson County, Tennessee'],data['Sumner County, Tennessee'],
                   data['Davidson County, Tennessee'],data['Wilson County, Tennessee'],
                   data['Trousdale County, Tennessee'],data['Williamson County, Tennessee'],
                   data['Rutherford County, Tennessee'],data['Maury County, Tennessee']]
data['GNRC Region'] = sum(GNRCCountiesAll)
MPOCounties = [data['Robertson County, Tennessee'],data['Sumner County, Tennessee'],
               data['Davidson County, Tennessee'],data['Wilson County, Tennessee'],
               data['Williamson County, Tennessee'],data['Rutherford County, Tennessee'],
               data['Maury County, Tennessee']]
data['MPO'] = sum(MPOCounties)
data = data.transpose().reset_index()

In [117]:
#make the county column the same as the name for the full counties.. and state whatever
data['County'] = data['NAME']

In [118]:
#replace the crime rate per 1,000 population so that it calculates correctly for the regions as well
data['Crime Rate (per 1,000)'] = (data['Number of Crimes']/data['Estimated Population']) * 1000

In [119]:
data = pd.concat([data, aside])

In [120]:
data['Year'] = '2022'

In [121]:
data.head()

,NAME,"Crime Rate (per 1,000)",Estimated Population,Number of Crimes,County,Year
0,Tennessee,71.891016,7051813.0,506962.0,Tennessee,2022
1,"Cheatham County, Tennessee",43.940114,42080.0,1849.0,"Cheatham County, Tennessee",2022
2,"Davidson County, Tennessee",115.769164,694520.0,80404.0,"Davidson County, Tennessee",2022
3,"Maury County, Tennessee",67.914241,108210.0,7349.0,"Maury County, Tennessee",2022
4,"Robertson County, Tennessee",37.26593,75565.0,2816.0,"Robertson County, Tennessee",2022


In [122]:
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'Tennessee Bureau of Investigation'

In [123]:
cols = ['Crime Rate (per 1,000)', 'Estimated Population', 'Number of Crimes']
data[cols] = data[cols].astype(float)

In [124]:
#final check
data.head()

,NAME,"Crime Rate (per 1,000)",Estimated Population,Number of Crimes,County,Year,GEO_ID,Source
0,Tennessee,71.891016,7051813.0,506962.0,Tennessee,2022,0400000US47,Tennessee Bureau of Investigation
1,"Cheatham County, Tennessee",43.940114,42080.0,1849.0,"Cheatham County, Tennessee",2022,0500000US47021,Tennessee Bureau of Investigation
2,"Davidson County, Tennessee",115.769164,694520.0,80404.0,"Davidson County, Tennessee",2022,0500000US47037,Tennessee Bureau of Investigation
3,"Maury County, Tennessee",67.914241,108210.0,7349.0,"Maury County, Tennessee",2022,0500000US47119,Tennessee Bureau of Investigation
4,"Robertson County, Tennessee",37.265930,75565.0,2816.0,"Robertson County, Tennessee",2022,0500000US47147,Tennessee Bureau of Investigation


In [125]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 141 entries, 0 to 137
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   NAME                    141 non-null    object 
 1   Crime Rate (per 1,000)  73 non-null     float64
 2   Estimated Population    82 non-null     float64
 3   Number of Crimes        102 non-null    float64
 4   County                  141 non-null    object 
 5   Year                    141 non-null    object 
 6   GEO_ID                  15 non-null     object 
 7   Source                  141 non-null    object 
dtypes: float64(3), object(5)
memory usage: 9.9+ KB


In [126]:
twentytwo = data

In [73]:
# #export to the SQLite database as only the JobsEQ annual data
# conn = sq.connect('../../Outputs/Dem_Transpo_Housing_Collection.db')
# data.to_sql('TBI_CrimeRate_2021', conn, if_exists = 'replace', index = False)
#export to the SQLite database as only the JobsEQ annual data
conn = sq.connect('../../Outputs/Dem_Transpo_Housing_Collection.db')
data.to_sql('TBI_CrimeRate_2022', conn, if_exists = 'replace', index = False)

141

In [18]:
#import and examine
data = pd.read_csv('../../Data Downloads/TBI_CrimeTypebyJurisdiction_IncidentDate2021.csv')
data.head(3)

,NAME,County,Murder,Negligent Manslaughter,Negligent Vehicular Manslaughter,Kidnapping/Abduction,Forcible Rape,Forcible Sodomy,Sexual Assault W/Object,Forcible Fondling,Incest,Statutory Rape,Aggravated Assault,Simple Assault,Intimidation,Stalking,Commercial Sex Acts,Involuntary Servitude,Arson,Bribery,Burglary,Counterfeiting/Forgery,Destruction/Damage/Vandalism,Embezzlement,Extortion/Blackmail,Fraud - False Pretenses,Fraud - Credit Card/ATM,Fraud - Impersonation,Fraud - Welfare,Fraud - Wire,Fraud - Identity Theft,Fraud - Computer Hacking/Invasion,Robbery,Theft - Pocket-picking,Theft - Purse Snatching,Theft - Shoplifting,Theft From Building,Theft From Coin Machine,Theft From Motor Vehicle,Theft of Motor Vehicle Parts,Theft - All Other Larceny,Motor Vehicle Theft,Stolen Property Offenses,Animal Cruelty,Drug/Narcotic Violations,Drug/Narcotic Equipment Violations,Gambling - Betting/Wagering,Gambling - Operating/Promoting,Gambling - Equipment Violations,Gambling - Sports Tampering,Pornography/Obscene Material,Prostitution,Prostitution Assisting/Promoting,Purchasing Prostitution,Weapon Law Violations
0,Tennessee,NaN,704.0,44.0,21.0,2206.0,2373.0,457.0,289.0,2744.0,27.0,367.0,38501.0,74668.0,25087.0,1847.0,118.0,2.0,955.0,26.0,23027.0,4795.0,46355.0,1577.0,630.0,12855.0,7118.0,3994.0,184.0,1629.0,441.0,166.0,4961.0,476.0,149.0,27335.0,16324.0,207.0,29676.0,13216.0,27883.0,21597.0,2806.0,588.0,55724.0,35045.0,9.0,14.0,9.0,NaN,1048.0,246.0,28.0,135.0,16030.0
1,Cheatham,NaN,2.0,NaN,NaN,8.0,7.0,NaN,NaN,9.0,NaN,1.0,86.0,268.0,79.0,7.0,NaN,NaN,1.0,NaN,58.0,35.0,156.0,7.0,NaN,56.0,17.0,33.0,NaN,22.0,NaN,NaN,3.0,NaN,NaN,59.0,75.0,1.0,46.0,54.0,96.0,59.0,18.0,2.0,431.0,280.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,55.0
2,Cheatham County Sheriff's Office,Cheatham,2.0,NaN,NaN,7.0,6.0,NaN,NaN,6.0,NaN,NaN,68.0,210.0,58.0,5.0,NaN,NaN,1.0,NaN,38.0,15.0,110.0,1.0,NaN,12.0,11.0,28.0,NaN,22.0,NaN,NaN,2.0,NaN,NaN,8.0,61.0,NaN,31.0,41.0,54.0,42.0,15.0,2.0,234.0,137.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,34.0


In [95]:
#import and examine
data = pd.read_csv('../../Data Downloads/TBI_CrimeTypebyJurisdiction_IncidentDate2022.csv')
data.head(3)

,NAME,County,Murder,Negligent Manslaughter,Negligent Vehicular Manslaughter,Kidnapping/Abduction,Forcible Rape,Forcible Sodomy,Sexual Assault W/Object,Forcible Fondling,Incest,Statutory Rape,Aggravated Assault,Simple Assault,Intimidation,Stalking,Commercial Sex Acts,Involuntary Servitude,Arson,Bribery,Burglary,Counterfeiting/Forgery,Destruction/Damage/Vandalism,Embezzlement,Extortion/Blackmail,Fraud - False Pretenses,Fraud - Credit Card/ATM,Fraud - Impersonation,Fraud - Welfare,Fraud - Wire,Fraud - Identity Theft,Fraud - Computer Hacking/Invasion,Robbery,Theft - Pocket-picking,Theft - Purse Snatching,Theft - Shoplifting,Theft From Building,Theft From Coin Machine,Theft From Motor Vehicle,Theft of Motor Vehicle Parts,Theft - All Other Larceny,Motor Vehicle Theft,Stolen Property Offenses,Crimes Against Society,Animal Cruelty,Drug/Narcotic Violations,Drug/Narcotic Equipment Violations,Gambling - Betting/Wagering,Gambling - Operating/Promoting,Gambling - Equipment Violations,Gambling - Sports Tampering,Pornography/Obscene Material,Prostitution,Prostitution Assisting/Promoting,Purchasing Prostitution,Weapon Law Violations
0,Tennessee,Full,650.0,48.0,14.0,1977.0,2253.0,506.0,231.0,3043.0,19.0,337.0,36982.0,73425.0,25128.0,1875.0,85.0,6.0,897.0,20.0,22373.0,5765.0,46774.0,2058.0,935.0,13317.0,7495.0,4325.0,62.0,1881.0,585.0,173.0,4891.0,787.0,141.0,28587.0,16186.0,231.0,31868.0,11874.0,28739.0,26655.0,2522.0,101242.0,648.0,51585.0,31114.0,16.0,24.0,20.0,NaN,1164.0,233.0,46.0,228.0,16164.0
1,Cheatham,Full,NaN,NaN,NaN,5.0,4.0,1.0,NaN,13.0,NaN,1.0,103.0,269.0,94.0,5.0,NaN,NaN,4.0,NaN,55.0,33.0,106.0,10.0,3.0,29.0,26.0,37.0,1.0,17.0,1.0,NaN,6.0,NaN,NaN,72.0,55.0,NaN,62.0,32.0,101.0,67.0,7.0,630.0,NaN,354.0,229.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,44.0
2,Cheatham County Sheriff's Office,Cheatham,NaN,NaN,NaN,2.0,4.0,1.0,NaN,8.0,NaN,1.0,68.0,204.0,72.0,4.0,NaN,NaN,3.0,NaN,36.0,21.0,78.0,6.0,3.0,9.0,10.0,24.0,1.0,16.0,NaN,NaN,1.0,NaN,NaN,5.0,37.0,NaN,37.0,21.0,60.0,43.0,4.0,289.0,NaN,164.0,99.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,23.0


In [96]:
#create a dictionary to rename the counties to full census names
countydict = {'Cheatham': 'Cheatham County, Tennessee', 'Davidson': 'Davidson County, Tennessee', 'Maury': 'Maury County, Tennessee', 
              'Robertson': 'Robertson County, Tennessee', 'Rutherford': 'Rutherford County, Tennessee', 'Sumner': 'Sumner County, Tennessee', 
              'Trousdale': 'Trousdale County, Tennessee', 'Williamson': 'Williamson County, Tennessee', 'Wilson': 'Wilson County, Tennessee', 
              'Dickson': 'Dickson County, Tennessee', 'Houston': 'Houston County, Tennessee', "Humphreys": 'Humphreys County, Tennessee',
              'Montgomery': 'Montgomery County, Tennessee', 'Stewart': 'Stewart County, Tennessee'}

In [97]:
data = data.set_index('County').transpose()
data = data.rename(columns = countydict)
data = data.transpose().reset_index()

In [98]:
aside = data.loc[data['County'] != 'Full']
data = data.loc[data['County'] == 'Full'].drop(columns = 'County')

In [99]:
#transpose to aggregate regions, then transpose back
data = data.set_index('NAME').transpose()
data = data.rename(columns = countydict)
GNRCCounties = [data['Stewart County, Tennessee'],data['Montgomery County, Tennessee'],
                data['Houston County, Tennessee'],data['Humphreys County, Tennessee'],
                data['Dickson County, Tennessee'],data['Cheatham County, Tennessee'],
                data['Robertson County, Tennessee'],data['Sumner County, Tennessee'],
                data['Davidson County, Tennessee'],data['Wilson County, Tennessee'],
                data['Trousdale County, Tennessee'],data['Williamson County, Tennessee'],
                data['Rutherford County, Tennessee']]
data['GNRC'] = sum(GNRCCounties)
GNRCCountiesAll = [data['Stewart County, Tennessee'],data['Montgomery County, Tennessee'],
                   data['Houston County, Tennessee'],data['Humphreys County, Tennessee'],
                   data['Dickson County, Tennessee'],data['Cheatham County, Tennessee'],
                   data['Robertson County, Tennessee'],data['Sumner County, Tennessee'],
                   data['Davidson County, Tennessee'],data['Wilson County, Tennessee'],
                   data['Trousdale County, Tennessee'],data['Williamson County, Tennessee'],
                   data['Rutherford County, Tennessee'],data['Maury County, Tennessee']]
data['GNRC Region'] = sum(GNRCCountiesAll)
MPOCounties = [data['Robertson County, Tennessee'],data['Sumner County, Tennessee'],
               data['Davidson County, Tennessee'],data['Wilson County, Tennessee'],
               data['Williamson County, Tennessee'],data['Rutherford County, Tennessee'],
               data['Maury County, Tennessee']]
data['MPO'] = sum(MPOCounties)
data = data.transpose().reset_index()

In [100]:
#make the county column the same as the name for the full counties.. and state whatever
data['County'] = data['NAME']
data = pd.concat([data, aside])

In [101]:
#set the index to name and county to fill the null values as 0 for mathematical operations
data = data.set_index(['NAME', 'County'])

In [102]:
data.fillna(0, inplace = True)

In [103]:
#aggregate crime type groups then drop the originals
thelist = [data['Negligent Manslaughter'], data['Negligent Vehicular Manslaughter']]
data['Manslaughter'] = sum(thelist)
thelist = [data['Forcible Rape'], data['Forcible Sodomy'], data['Sexual Assault W/Object'], data['Forcible Fondling'], data['Incest'], data['Statutory Rape'], 
          data['Aggravated Assault'], data['Simple Assault']]
data['Assault & Violent Sex Crime'] = sum(thelist)
thelist = [data['Intimidation'], data['Stalking']]
data['Intimidation & Stalking'] = sum(thelist)
thelist = [data['Arson'], data['Destruction/Damage/Vandalism']]
data['Arson, Destruction & Vandalism'] = sum(thelist)
thelist = [data['Fraud - False Pretenses'], data['Fraud - Credit Card/ATM'], data['Fraud - Impersonation'], data['Fraud - Welfare'], data['Fraud - Wire'], 
          data['Fraud - Identity Theft'], data['Fraud - Computer Hacking/Invasion']]
data['Fraud'] = sum(thelist)
thelist = [data['Theft - Pocket-picking'], data['Theft - Purse Snatching'], data['Theft - Shoplifting'], data['Theft From Building'], 
           data['Theft From Coin Machine'], data['Theft From Motor Vehicle'], data['Theft of Motor Vehicle Parts'], data['Theft - All Other Larceny'], 
           data['Motor Vehicle Theft'], data['Stolen Property Offenses']]
data['Theft'] = sum(thelist)
thelist = [data['Drug/Narcotic Violations'], data['Drug/Narcotic Equipment Violations']]
data['Drugs & Narcotics Violations'] = sum(thelist)
thelist = [data['Bribery'], data['Counterfeiting/Forgery'], data['Embezzlement'], data['Extortion/Blackmail'],
           data['Gambling - Betting/Wagering'], data['Gambling - Operating/Promoting'], data['Gambling - Equipment Violations'], 
           data['Gambling - Sports Tampering']]
data['Bribery, Forgery, Blackmail, Embezzlement & Gambling'] = sum(thelist)
thelist = [data['Prostitution'], data['Prostitution Assisting/Promoting'], data['Purchasing Prostitution'], 
           data['Commercial Sex Acts'], data['Pornography/Obscene Material']]
data['Prostitution, Commercial Sex Acts & Pornography, Aiding & Purchasing Prostitution'] = sum(thelist)
cols = ['Negligent Manslaughter','Negligent Vehicular Manslaughter','Forcible Rape','Forcible Sodomy','Sexual Assault W/Object','Forcible Fondling','Incest',
        'Statutory Rape','Aggravated Assault','Simple Assault','Intimidation','Stalking','Arson','Destruction/Damage/Vandalism','Bribery','Counterfeiting/Forgery',
        'Embezzlement','Extortion/Blackmail','Fraud - False Pretenses','Fraud - Credit Card/ATM','Fraud - Impersonation','Fraud - Welfare','Fraud - Wire',
        'Fraud - Identity Theft','Fraud - Computer Hacking/Invasion', 'Theft - Pocket-picking','Theft - Purse Snatching','Theft - Shoplifting','Theft From Building',
        'Theft From Coin Machine','Theft From Motor Vehicle','Theft of Motor Vehicle Parts','Theft - All Other Larceny','Motor Vehicle Theft',
        'Commercial Sex Acts', 'Pornography/Obscene Material',
        'Stolen Property Offenses','Drug/Narcotic Violations','Drug/Narcotic Equipment Violations','Gambling - Betting/Wagering','Gambling - Operating/Promoting',
        'Gambling - Equipment Violations','Gambling - Sports Tampering','Prostitution','Prostitution Assisting/Promoting','Purchasing Prostitution', 
        'Involuntary Servitude']# there is just hardly any of this I am dropping it for this analysis
data = data.drop(columns = cols)

In [104]:
#reset the index and examine
data = data.reset_index()

In [105]:
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'Tennessee Bureau of Investigation'

In [106]:
#final check
data.head()

,NAME,County,Murder,Kidnapping/Abduction,Burglary,Robbery,Crimes Against Society,Animal Cruelty,Weapon Law Violations,Manslaughter,Assault & Violent Sex Crime,Intimidation & Stalking,"Arson, Destruction & Vandalism",Fraud,Theft,Drugs & Narcotics Violations,"Bribery, Forgery, Blackmail, Embezzlement & Gambling","Prostitution, Commercial Sex Acts & Pornography, Aiding & Purchasing Prostitution",GEO_ID,Source
0,Tennessee,Tennessee,650.0,1977.0,22373.0,4891.0,101242.0,648.0,16164.0,62.0,116796.0,27003.0,47671.0,27838.0,147590.0,82699.0,8838.0,1756.0,0400000US47,Tennessee Bureau of Investigation
1,"Cheatham County, Tennessee","Cheatham County, Tennessee",0.0,5.0,55.0,6.0,630.0,0.0,44.0,0.0,391.0,99.0,110.0,111.0,396.0,583.0,46.0,3.0,0500000US47021,Tennessee Bureau of Investigation
2,"Davidson County, Tennessee","Davidson County, Tennessee",117.0,245.0,3626.0,1374.0,13670.0,21.0,4495.0,2.0,18819.0,4061.0,7285.0,4746.0,25227.0,8958.0,1218.0,209.0,0500000US47037,Tennessee Bureau of Investigation
3,"Maury County, Tennessee","Maury County, Tennessee",4.0,34.0,230.0,31.0,1379.0,11.0,129.0,0.0,2176.0,432.0,619.0,605.0,1678.0,1221.0,160.0,19.0,0500000US47119,Tennessee Bureau of Investigation
4,"Robertson County, Tennessee","Robertson County, Tennessee",5.0,9.0,69.0,4.0,1030.0,4.0,60.0,0.0,850.0,93.0,183.0,56.0,480.0,962.0,37.0,4.0,0500000US47147,Tennessee Bureau of Investigation


In [107]:
data['Year'] = '2022'

In [108]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 21 columns):
 #   Column                                                                             Non-Null Count  Dtype  
---  ------                                                                             --------------  -----  
 0   NAME                                                                               141 non-null    object 
 1   County                                                                             141 non-null    object 
 2   Murder                                                                             282 non-null    float64
 3   Kidnapping/Abduction                                                               282 non-null    float64
 4   Burglary                                                                           282 non-null    float64
 5   Robbery                                                                            282 non-null    float64

In [109]:
#export to feather to join to all tidydata
data.to_feather('../../Outputs/TBI_CrimeType_2022')
#export to the SQLite database
conn = sq.connect('../../Outputs/Dem_Transpo_Housing_Collection.db')
data.to_sql('TBI_CrimeType_2022', conn, if_exists = 'replace', index = False)

282

In [130]:
#import and examine data
data = pd.read_csv('../../Data Downloads/TBI_CrimeRatesbyJurisdiction_05to22.csv')
data.head(3)

,County,NAME,Year,"Crime Rate (per 1,000)",Estimated Population,Number of Crimes
0,NaN,Tennessee,2001,101.47,"5,755,443","583,990"
1,NaN,Tennessee,2002,101.49,"5,803,306","588,969"
2,NaN,Tennessee,2003,103.36,"5,856,522","605,315"


In [128]:
twentytwo = twentytwo.drop(columns = ['GEO_ID', 'Source'])

In [129]:
twentytwo.head()

,NAME,"Crime Rate (per 1,000)",Estimated Population,Number of Crimes,County,Year
0,Tennessee,71.891016,7051813.0,506962.0,Tennessee,2022
1,"Cheatham County, Tennessee",43.940114,42080.0,1849.0,"Cheatham County, Tennessee",2022
2,"Davidson County, Tennessee",115.769164,694520.0,80404.0,"Davidson County, Tennessee",2022
3,"Maury County, Tennessee",67.914241,108210.0,7349.0,"Maury County, Tennessee",2022
4,"Robertson County, Tennessee",37.265930,75565.0,2816.0,"Robertson County, Tennessee",2022


In [131]:
data = pd.concat([data, twentytwo])

In [132]:
conn = sq.connect('../../Outputs/Dem_Transpo_Housing_Collection.db')
data.to_sql('TBI_CrimeRatesbyJurisdiction_05to22', conn, if_exists = 'replace', index = False)

15497